# conduct Kfold test and feature importance on training data

### What does this notebook do?

This notebook takes a CSV of training data and conducts a self built k-fold validation on it using the sklearn.ensemble  ExtraTreesClassifier. It produces a scroe for the average % missclassifed an infromation on what the feature importance was for each trained model. 10 models are trained on different sub-sets of the data, 9/10 chunks are used each time then used to classify the remaining 1/10 chunk of data. 


### Requierments 

You need to run the following commands from the command line prior to launching jupyter notebook from the same terminal so that the required libraries and paths are set.

    $ module use /g/data/v10/public/modules/modulefiles

    $ module load dea


In [1]:
import datacube
from datacube.storage import masking
from datacube import Datacube
from datetime import datetime
from skimage import exposure

#load requiered modules #load r 
import numpy as np
import rasterio
from rasterio import features

import xarray as xr
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame

import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.ensemble import ExtraTreesClassifier

In [2]:
import glue

In [3]:
from sklearn.datasets import make_classification

In [4]:
#define fuction for loading part of a bulti band GIF 

def load_gfif_full(file_name, x=1):


    dataset = rasterio.open(file_name) #open using Raterio
    {i: dtype for i, dtype in zip(dataset.indexes, dataset.dtypes)}
    data_array = dataset.read(x) #read selected band into np array
    data_xr = xr.DataArray(data_array) #turn into Xr array for fuctionality #Now don't need xr yet
    return(data_xr) #return np array into program 

In [8]:
row_number = 538070


In [10]:
panda_training= pd.read_csv('/g/data/u46/users/ck9738/Datasets/ML_for_smad_green/Glueviz_datasets/aus_alps_traning_10_small_even.csv'
                           ,nrows=row_number)#
# open the data we are going to use for training and testing as a dataframe
panda_training

,Unnamed: 0,Band 1,Band 2,Band 3,Band 4,Band 5,Band 6,Band 7,Band 8,Band 9,Band 10,class
0,1534095,256.0,405.0,442.0,759.0,1380.0,1565.0,1705.0,1800.0,1636.0,1027.0,1
1,5340326,485.0,748.0,854.0,1358.0,2259.0,2567.0,2838.0,2974.0,2813.0,1757.0,0
2,1425858,372.0,506.0,608.0,935.0,1635.0,1860.0,1949.0,2133.0,2013.0,1198.0,1
3,1153896,222.0,321.0,330.0,594.0,1076.0,1242.0,1366.0,1418.0,1277.0,756.0,1
4,4366429,700.0,1057.0,1288.0,1654.0,2359.0,2538.0,2806.0,2826.0,3225.0,2204.0,0
5,1525203,387.0,579.0,671.0,900.0,1590.0,1812.0,2074.0,2053.0,2104.0,1274.0,1
6,5073710,492.0,703.0,897.0,1211.0,1905.0,2162.0,2248.0,2472.0,2385.0,1514.0,0
7,757797,261.0,395.0,433.0,743.0,1375.0,1556.0,1656.0,1804.0,1591.0,968.0,1
8,734423,314.0,444.0,485.0,836.0,1444.0,1725.0,1922.0,1956.0,1896.0,1197.0,1
9,342826,196.0,320.0,313.0,663.0,1633.0,1920.0,2000.0,2148.0,1257.0,649.0,1


In [11]:
blue_list = panda_training['Band 1'].values.tolist()
green_list = panda_training['Band 2'].values.tolist()
red_list = panda_training['Band 3'].values.tolist()
rede1_list = panda_training['Band 4'].values.tolist()
rede2_list = panda_training['Band 5'].values.tolist()
rede3_list = panda_training['Band 6'].values.tolist()
nir1_list = panda_training['Band 7'].values.tolist()
nir2_list = panda_training['Band 8'].values.tolist()
swir1_list = panda_training['Band 9'].values.tolist()
swir2_list = panda_training['Band 10'].values.tolist()

classification = panda_training['class'].values.tolist()
#create array of features to use in classifier and array of the classification answer
how_many = len(red_list)
how_many

features = []

for i in range(how_many):
    features.append((blue_list[i],
                    green_list[i],
                    red_list[i],
                    rede1_list[i],
                    rede2_list[i],
                    rede3_list[i],
                    nir1_list[i],
                    nir2_list[i],
                    swir1_list[i],
                    swir2_list[i]))
features
features_array = np.asarray(features)

classification_array = np.asarray(classification)

In [12]:

# aero_list = panda_training['t_aerosol'].values.tolist()
# red_list = panda_training['t_red'].values.tolist()
# blue_list = panda_training['t_blue'].values.tolist()
# green_list = panda_training['t_green'].values.tolist()
# rede1_list = panda_training['t_rededge1'].values.tolist()
# rede2_list = panda_training['t_rededge2'].values.tolist()
# rede3_list = panda_training['t_rededge3'].values.tolist()
# nir1_list = panda_training['t_nir1'].values.tolist()
# nir2_list = panda_training['t_nir2'].values.tolist()
# swir1_list = panda_training['t_swir1'].values.tolist()
# swir2_list = panda_training['t_swir2'].values.tolist()

# classification = panda_training['class'].values.tolist()

# #create array of features to use in classifier and array of the classification answer
# how_many = len(red_list)
# how_many

# features = []

# for i in range(how_many):
#     features.append((
#                     aero_list[i],
#                     blue_list[i],
#                     green_list[i],
#                     red_list[i],
#                     rede1_list[i],
#                     rede2_list[i],
#                     rede3_list[i],
#                     nir1_list[i],
#                     nir2_list[i],
#                     swir1_list[i],
#                     swir2_list[i],)

# features
# features_array = np.asarray(features)

# classification_array = np.asarray(classification)

In [13]:
features_array

array([[ 256.,  405.,  442., ..., 1800., 1636., 1027.],
       [ 485.,  748.,  854., ..., 2974., 2813., 1757.],
       [ 372.,  506.,  608., ..., 2133., 2013., 1198.],
       ...,
       [ 467.,  682.,  781., ..., 2741., 2579., 1644.],
       [ 276.,  403.,  445., ..., 1665., 1665.,  982.],
       [ 502.,  716.,  930., ..., 2223., 3035., 2000.]])

In [14]:
classification_array

array([1, 0, 1, ..., 0, 1, 0])

In [15]:
#Split the feature array and the classification answer arrays into kn chuncks for the kfold validation. here K = 10

K=10

kt1,kt2,kt3,kt4,kt5,kt6,kt7,kt8,kt9,kt10 = np.split(features_array, 10)
kc1,kc2,kc3,kc4,kc5,kc6,kc7,kc8,kc9,kc10 = np.split(classification_array, 10)

In [16]:
#define classifier

#pick your method

classify = ExtraTreesClassifier(n_estimators=250, max_depth=30, bootstrap=True, random_state=0)

In [17]:
#define classifily fuction 

def train_and_test(test_f,train_f,train_c):
    """    takes data in chunks, puts the training data back into a stack to train and putputs a 
    boolian array of the test cassification
    test_f = test data to be classified. 2D array size[sample_n, features_n]
    
    train_f = list of arrays each 2D array size[sample_n, features_n].
            each is one K chunck of the total data set, to be used to train. should equal total data - 1k.
    
    train_C = list of arrays each 1D array size[sample_n]. boolean values equal to classification answer
            each is one K chunck of the total data set, to be used to train. should equal total data - 1k.
            
    returns = 1D array size[sample_n]. boolean values equal to classification answer. size equal to test_f
    """
    train_f = np.concatenate(train_f,axis=0)
    train_c = np.concatenate(train_c,axis=0)
    classify.fit(train_f,train_c)#fit the moddle to the training data
    results=classify.predict(test_f)#run predict on test data
    importance = classify.feature_importances_
    print(importance)
    return(results,importance)


In [18]:
result_one,import_one = train_and_test(kt1,[kt2,kt3,kt4,kt5,kt6,kt7,kt8,kt9,kt10],
                                [kc2,kc3,kc4,kc5,kc6,kc7,kc8,kc9,kc10])
result_two,import_two = train_and_test(kt2,[kt1,kt3,kt4,kt5,kt6,kt7,kt8,kt9,kt10],
                                [kc1,kc3,kc4,kc5,kc6,kc7,kc8,kc9,kc10])
result_three,import_three = train_and_test(kt3,[kt2,kt1,kt4,kt5,kt6,kt7,kt8,kt9,kt10],
                                [kc2,kc1,kc4,kc5,kc6,kc7,kc8,kc9,kc10])
result_four,import_four = train_and_test(kt4,[kt2,kt3,kt1,kt5,kt6,kt7,kt8,kt9,kt10],
                                [kc2,kc3,kc1,kc5,kc6,kc7,kc8,kc9,kc10])
result_five,import_five = train_and_test(kt5,[kt2,kt3,kt4,kt1,kt6,kt7,kt8,kt9,kt10],
                                [kc2,kc3,kc4,kc1,kc6,kc7,kc8,kc9,kc10])
result_six,import_six = train_and_test(kt6,[kt2,kt3,kt4,kt5,kt1,kt7,kt8,kt9,kt10],
                                [kc2,kc3,kc4,kc5,kc1,kc7,kc8,kc9,kc10])
result_seven,import_seven = train_and_test(kt7,[kt2,kt3,kt4,kt5,kt6,kt1,kt8,kt9,kt10],
                                [kc2,kc3,kc4,kc5,kc6,kc1,kc8,kc9,kc10])
result_eight,import_eight = train_and_test(kt8,[kt2,kt3,kt4,kt5,kt6,kt7,kt1,kt9,kt10],
                                [kc2,kc3,kc4,kc5,kc6,kc7,kc1,kc9,kc10])
result_nine,import_nine = train_and_test(kt9,[kt2,kt3,kt4,kt5,kt6,kt7,kt8,kt1,kt10],
                                [kc2,kc3,kc4,kc5,kc6,kc7,kc8,kc1,kc10])
result_ten,import_ten = train_and_test(kt10,[kt2,kt3,kt4,kt5,kt6,kt7,kt8,kt9,kt1],
                                [kc2,kc3,kc4,kc5,kc6,kc7,kc8,kc9,kc1])

[0.05815209 0.10676463 0.08728594 0.12415048 0.08230737 0.06172542
 0.05686952 0.06825067 0.18023767 0.17425622]
[0.04862368 0.10151099 0.08816041 0.15354534 0.07716409 0.0705495
 0.04841298 0.063304   0.18897797 0.15975103]
[0.04829593 0.1010371  0.07499969 0.14695078 0.08431254 0.05644484
 0.04983695 0.07192839 0.19237809 0.1738157 ]
[0.04244873 0.10579727 0.08274899 0.14044981 0.0841487  0.06205687
 0.05837049 0.07094625 0.17695503 0.17607786]
[0.04563364 0.11223317 0.08196169 0.13921285 0.08359885 0.06218844
 0.05098683 0.06522847 0.1885313  0.17042475]
[0.03671125 0.09995558 0.09437949 0.13058671 0.09901702 0.05199361
 0.05246289 0.07206223 0.1842089  0.17862232]
[0.05527356 0.10452269 0.08676237 0.14069497 0.08883965 0.06243028
 0.05145257 0.06339143 0.17721118 0.1694213 ]
[0.04330851 0.10361626 0.08564831 0.1475065  0.09089937 0.06298813
 0.05234739 0.06165425 0.18113215 0.17089914]
[0.0397814  0.10175821 0.09303527 0.1410584  0.08726741 0.06771448
 0.05076538 0.06735941 0.18016

In [19]:

#compleate test for each K chunk
join_results = np.concatenate((result_one,result_two,result_three,result_four,result_five,result_six,
                               result_seven,result_eight,result_nine,result_ten),axis=0)


In [20]:
classify.get_params

<bound method BaseEstimator.get_params of ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=30, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)>

In [21]:
panda_training['test_result']=pd.Series(join_results, index=panda_training.index)
#add results as new column to panda data frame


In [22]:
panda_training

,Unnamed: 0,Band 1,Band 2,Band 3,Band 4,Band 5,Band 6,Band 7,Band 8,Band 9,Band 10,class,test_result
0,1534095,256.0,405.0,442.0,759.0,1380.0,1565.0,1705.0,1800.0,1636.0,1027.0,1,1
1,5340326,485.0,748.0,854.0,1358.0,2259.0,2567.0,2838.0,2974.0,2813.0,1757.0,0,0
2,1425858,372.0,506.0,608.0,935.0,1635.0,1860.0,1949.0,2133.0,2013.0,1198.0,1,1
3,1153896,222.0,321.0,330.0,594.0,1076.0,1242.0,1366.0,1418.0,1277.0,756.0,1,1
4,4366429,700.0,1057.0,1288.0,1654.0,2359.0,2538.0,2806.0,2826.0,3225.0,2204.0,0,0
5,1525203,387.0,579.0,671.0,900.0,1590.0,1812.0,2074.0,2053.0,2104.0,1274.0,1,1
6,5073710,492.0,703.0,897.0,1211.0,1905.0,2162.0,2248.0,2472.0,2385.0,1514.0,0,0
7,757797,261.0,395.0,433.0,743.0,1375.0,1556.0,1656.0,1804.0,1591.0,968.0,1,1
8,734423,314.0,444.0,485.0,836.0,1444.0,1725.0,1922.0,1956.0,1896.0,1197.0,1,1
9,342826,196.0,320.0,313.0,663.0,1633.0,1920.0,2000.0,2148.0,1257.0,649.0,1,1


In [23]:
difference = np.logical_not(classification_array==join_results) #will be True where not correctly classified
difference = difference*1 #turn all True into 1
difference

array([0, 0, 0, ..., 0, 0, 0])

In [26]:
number_correct_total = np.count_nonzero(difference) #count the number of incorrect classifications

total_percent = (number_correct_total/row_number)*100#calculate what total percent wast incorrectly classified
print('in total ', number_correct_total,'out of ',row_number,'where incorrectly classified')
print('this is',total_percent,'%')

in total  882 out of  538070 where incorrectly classified
this is 0.1639191926700987 %
